In [1]:
import torch
import utilities
import numpy as np
import pandas as pd
from sc_sharp import scSHARP

/home/users/lewinsda/.conda/envs/daniel_thesis_2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
x = np.array([[1,1], [2,2], [3,3], [10,10], [11,11], [12,12]])
preds = [0, -1, 0, 1, -1, 1]

In [17]:
utilities.knn_consensus(x, preds, 2, one_epoch=True)

array([0, 0, 0, 1, 1, 1])

In [18]:
utilities.knn_consensus_batch(x, preds, 2, batch_size = 3, converge=True)

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


array([0., 0., 0., 1., 1., 1.], dtype=float32)

In [2]:
#data_path = "/home/groups/ConradLab/daniel/sharp_data/sharp_sims_test/splat_0.7_de_rq/"
data_path = "/home/groups/ConradLab/daniel/sharp_data/sharp_sims_test/splat_0.8_de_rq/"
#data_path = "/home/groups/ConradLab/daniel/sharp_data/jung/"

In [3]:
counts = pd.read_csv(data_path + "query_counts.csv", index_col = 0)
#counts = pd.read_csv(data_path + "counts.csv", index_col = 0)
counts.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene33685,Gene33686,Gene33687,Gene33688,Gene33689,Gene33690,Gene33691,Gene33692,Gene33693,Gene33694
Cell1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
all_labels = pd.read_csv(data_path + "preds.csv", index_col=0)
all_labels.head()

,scina,scsorter,sctype,singler,scpred
Cell1001,NaN,Group4,Group4,Group4,Group4
Cell1002,Group3,Group3,Group3,Group3,Group3
Cell1003,Group3,Group3,Group3,Group3,Group3
Cell1004,NaN,Group3,Group2,Group3,Group3
Cell1005,NaN,Group3,Group3,Group3,Group3


In [5]:
X, keep_cells,keep_genes,pca_obj = utilities.preprocess(np.array(counts), scale=False, run_pca=True)
X.shape

(998, 500)

In [6]:
_,marker_names = utilities.read_marker_file(data_path + "markers.txt")

In [7]:
all_labels_factored = utilities.factorize_df(all_labels, marker_names)
encoded_labels = utilities.encode_predictions(all_labels_factored)
confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = .51)
confident_labels = confident_labels

In [8]:
real_y = pd.read_csv(data_path + "query_meta.csv", index_col= 0).Group
#real_y = pd.read_csv(data_path + "labels.csv", index_col= 0).iloc[:,0]
real_y = pd.factorize(real_y, sort=True)[0]
real_y = real_y[keep_cells]

In [9]:
# one epoch
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus(X, confident_labels, i, converge=False, one_epoch=True)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])


10
(0.37374749779701233, array([[  0,   0,   0,   0,   0],
       [  0,  69,   0,   0, 185],
       [  1,   2,   1,   1, 239],
       [  1,   0,   0,  58, 196],
       [  0,   0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [10]:
# one epoch
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=True, batch_size=1000)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

10
(0.37374749779701233, array([[  0,   0,   0,   0,   0],
       [  0,  69,   0,   0, 185],
       [  1,   2,   1,   1, 239],
       [  1,   0,   0,  58, 196],
       [  0,   0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [27]:
# until all labelled
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus(X, confident_labels, i, converge=False, one_epoch=False)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1
2
10
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [11]:
# until all labelled
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=False, batch_size=1000)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


2
10
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [28]:
# until converge
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus(X, confident_labels, i, converge=True, one_epoch=False)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1
2
10
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
50
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
100
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
200
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
400
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
2
500
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))


In [16]:
# until converge batch
for i in [5,10,50,100]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, batch_size = 300, converge=True, one_epoch=False)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


5
(0.8786126971244812, array([[  0,   0,   0,   0],
       [  3, 167,  71,   3],
       [  1,   5, 150,   0],
       [  1,   0,   0, 291]]))
1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


10
(0.8771676421165466, array([[  0,   0,   0,   0],
       [  4, 163,  75,   2],
       [  0,   4, 152,   0],
       [  0,   0,   0, 292]]))
1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


50
(0.8352600932121277, array([[150,  92,   2],
       [ 16, 139,   1],
       [  0,   3, 289]]))
1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


2
100
(0.6994219422340393, array([[ 76, 133,  35],
       [ 17, 119,  20],
       [  2,   1, 289]]))


In [ ]:
preds = utilities.knn_consensus(X, confident_labels, 700, converge=True, one_epoch=False)

print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

In [ ]:
preds.shape

In [76]:
print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(confident_labels), range(len(real_y)), range(len(real_y)))[0:2])

(0.8078034520149231, array([[  0,   0,   0,   0],
       [ 32, 136,  74,   2],
       [  4,  10, 142,   0],
       [ 10,   1,   0, 281]]))
